# Artigo utilizado como base para a solução
https://research.ijcaonline.org/volume58/number17/pxc3883886.pdf

# Requisitos da atividade
1) implementar o gerador dos labirintos;

2) implementar o algoritmo genético (AG) com estruturas de dados e parâmetros necessários;

3) executar o AG num labirinto 10x10;

4) traçar um gráfico da evolução do AG (gerações versus fitness da população) usando a biblioteca matplotlib;

5) desenhar uma figura do labirinto e da melhor solução encontrada (pode ser em modo texto);

6) (Bonus) Variar os parâmetros do AG e comentar o efeito sobre a velocidade de convergência da solução.

### Grupo
* Alexandre Ribeiro
* Flávio Farias
* Henrique Arriel

# Imports e constantes

In [ ]:
import random
from random import randint, choice
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from time import sleep

SEED = 1123581321 #seed utilizada para a randomização ser igual em todas execuções do notebook. A modificação da seed pode trazer efeitos colaterais para algumas análises.
rep = { "cross": '#', 'vwall': '#', 'hwall': '#', 'step': '\033[95m+\033[00m', 'start': '\033[92mS\033[00m', 'finish': '\033[91mF\033[00m' }

# Gerador de labirintos (A-Mazer)

#### Gerar labirinto

In [ ]:
  def maze_maker(n):
    maze = [[[0,0,0,0,0] for line in range(n)] for column in range(n)]
    doors = ['LEFT', 'RIGHT', 'TOP', 'BOTTOM']
    for row in range(1,n-1):
      for col in range(1,n-1):
        for door in doors:
          opened_door = randint(0,1)
          if(opened_door):
            open_door(maze, (row, col), door)

    return maze

  def validate_cell(line,column, maze_size):
    return ((0 <= line < maze_size) and (0 <= column < maze_size))

  def start_finish_cells(maze_size):
    start = randint(0,maze_size-1)
    finish = randint(0,maze_size-1)
    return start, finish

  def open_door(maze, coordinates, door):
    x, y = coordinates
    if (door == "LEFT"):
      if(validate_cell(x, y, len(maze)) and validate_cell(x, y-1, len(maze))):
        maze[x][y][0], maze[x][y-1][1] = 1, 1
    elif (door == "RIGHT"):
      if(validate_cell(x, y, len(maze)) and validate_cell(x, y+1, len(maze))):
        maze[x][y][1], maze[x][y+1][0] = 1, 1
    elif (door == "TOP"): 
      if(validate_cell(x, y, len(maze)) and validate_cell(x-1, y, len(maze))):
        maze[x][y][2], maze[x-1][y][3] = 1, 1
    elif (door == "BOTTOM"): 
      if(validate_cell(x, y, len(maze)) and validate_cell(x+1, y, len(maze))):
        maze[x][y][3], maze[x+1][y][2] = 1, 1

#### Flood fill

In [ ]:
def flood_fill(maze, row, col, label):
  if(maze[row][col][4] == label):
    return
  else:
    maze[row][col][4] = label
    #LEFT
    if(maze[row][col][0] == 1): flood_fill(maze, row,col-1, label)
    #RIGHT
    if(maze[row][col][1] == 1): flood_fill(maze, row, col+1, label)
    #TOP
    if(maze[row][col][2] == 1): flood_fill(maze, row-1, col, label)
    #BOTTOM
    if(maze[row][col][3] == 1): flood_fill(maze, row+1, col, label)

#### Junção de paredes adjacentes

In [ ]:
#this function considers current and parent as neighbors
def came_from(current, parent):
  coord_to_text = { (0, 1): 'LEFT', (0, -1): 'RIGHT', (1, 0): 'TOP', (-1, 0): 'BOTTOM' }
  x, y = current
  px, py = parent
  from_ = (x-px, y-py)
  
  return coord_to_text[from_]

def text_to_step(text): 
  steps = {'LEFT': 0, 'RIGHT': 1, 'TOP': 2, 'BOTTOM': 3}
  return steps[text]

def step_to_text(step):
  texts = {0: 'LEFT', 1: 'RIGHT', 2: 'TOP', 3: 'BOTTOM'}
  return texts[step]

#this function doesnt consider whether door is open
def get_neighbors(maze, node):
  x, y = node
  doors = [0, 1, 2, 3]
  door_to_coord = { 0: (0, -1), 1: (0, 1), 2: (-1, 0), 3: (1, 0) }
  neighbors = []
  for door in doors:
      neighbor = (x + door_to_coord[door][0], y + door_to_coord[door][1])
      if (validate_cell(neighbor[0], neighbor[1], len(maze))):
        neighbors.append(neighbor)
  return neighbors

def merge_adjacent(maze, row, col):
  stack = []
  visited = set()
  parent = None
  stack.append(((row, col), parent))

  while(stack):
    current, parent = stack.pop()
    if (current in visited): continue
    if (maze[current[0]][current[1]][4] == 2):
      open_door(maze, current, came_from(current, parent))
      return True

    visited.add(current)
    neighbors = get_neighbors(maze, current)
    for neighbor in neighbors: 
      if (maze[neighbor[0]][neighbor[1]][4] != 0): 
        stack.append((neighbor, current))
  return False

#### Junção de paredes randômicas

In [ ]:
#get a random cell reachable from the start, inclusive
def random_cell(maze, start, except_=[]):
  stack = []
  visited = set()
  parent = None
  stack.append((start, parent))

  while(stack):
    current, parent = stack.pop()
    if (current in visited): continue
    visited.add(current)
    neighbors = get_neighbors(maze, current)
    for neighbor in neighbors: 
      if (maze[neighbor[0]][neighbor[1]][4] == 1): 
        stack.append((neighbor, current))

  [visited.remove(node) for node in except_ if node in visited]
  return choice(list(visited)) if visited else False

#get a valid random door
#cell e neighbors podem não ter uma porta entre si
def random_door(maze, cell):
  neighbors = get_neighbors(maze, cell)
  neighbor_cell = choice(neighbors)
  neighbors.remove(neighbor_cell)
  while(neighbors and maze[neighbor_cell[0]][neighbor_cell[1]][4]):
    neighbor_cell = choice(neighbors)
    neighbors.remove(neighbor_cell)
  
  return neighbor_cell if not maze[neighbor_cell[0]][neighbor_cell[1]][4] else False


def open_random_door(maze, start):
  cell = random_cell(maze, start)
  door = random_door(maze, cell)
  tried = [] #list of cells that there are not doors to open
  
  while(not door and cell):
    cell = random_cell(maze, start, tried)
    door = random_door(maze, cell)
  if (not cell): print("fully opened")
  else: 
    open_door(maze, cell, step_to_text(text_to_step(came_from(cell, door))))


#### Verifica e torna o labirinto viável

In [ ]:
def valid_maze(maze, maze_size, finish):
  return maze[finish][maze_size-1][4] == 1

def make_maze_viable(maze, start, finish):
  maze_size = len(maze)
  flood_fill(maze, start, 0, 1)

  if (not valid_maze(maze, maze_size, finish)): 
    flood_fill(maze, finish, maze_size-1, 2)

    while(not valid_maze(maze, maze_size, finish) ):

      merged = merge_adjacent(maze, start, 0)
      if (merged):
        flood_fill(maze, start, 0, 0)
        flood_fill(maze, start, 0, 1)
      else:
        open_random_door(maze, (start, 0))
        flood_fill(maze,start, 0, 0)
        flood_fill(maze, start, 0, 1)

# Algoritmo genético

# Execução do algoritmo genético em um labirinto 10x10

# Gráfico com evolução do algoritmo genético (gerações x fitness)

# Figura do labirinto e melhor solução encontrada

# Variação dos parâmetros do algoritmo genético

Para efeitos de comparação das variações de parâmetros do algoritmo genético, reiniciaremos a seed utilzada para gerarmos os mesmos labirintos para os diferentes parâmetros.

Parâmetros base

---
Taxa de crossover == **0.8**

Taxa de mutação == **0.1**

Tamanho da população == **100**

Geração máxima para parada == **500**

Esses parâmetros foram obtidos através do artigo e foram utilizados nas seções anteriores.

Para termos uma média de gerações de convergência do AG, rodamos o algoritmo randômicamente por 50 vezes. 

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  _, generation, _, _, _, _ = run(maze_size=10, population_size=100, crossover_rate=0.8, mutation_rate= 0.1, max_generation=500,n_generation_growth= 5)
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Parâmetros variados

----

Taxa de crossover == ~0.8~ => **0.6**

Taxa de mutação == ~0.1~ => **0.3**

Tamanho da população == ~100~ => **80**

Geração máxima para parada == ~500~ => **1500**

Seguindo a lógica acima, rodamos o algoritmo com os parâmetros variados mais 50 vezes. 

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  chromosome, generation, fitness_values, maze, _, _ = run(maze_size=10, population_size=80, crossover_rate=0.6, mutation_rate= 0.3, max_generation=1500,n_generation_growth= 5)
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Aparentemente, a média de gerações para a convergência do Algoritmo Genético com os parâmetros variados tendeu a diminuir.

O gráfico a seguir é referente à ultima execução do algoritmo.



In [ ]:
show_graph(generation, fitness_values)

#### Utilizando outra função de fitness (Distância de Manhattan)

No exemplo a seguir utilizaremos a função fitness com Distância de Manhattan ao invés da Distância Euclideana.

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  chromosome, generation, fitness_values, maze, _, _ = run(maze_size=10, population_size=100, crossover_rate=0.8, mutation_rate= 0.1, max_generation=500,n_generation_growth= 5, fitness_function= "MANHATTAN")
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Ao alterar a função fitness para utilizar a Distância de Manhattan, notamos uma piora na convergência do algoritmo genético com os mesmos parâmetros base (artigo).

O gráfico a seguir é referente à ultima execução do algoritmo.

In [ ]:
show_graph(generation, fitness_values)


#### Efeito da variação dos parâmetros na velocidade de convergência da solução


O algoritmo genético com a variação dos parâmetros e função fitness de Distância Euclidiana convergiu mais rapidamente que o algoritmo com os parâmetros base.

Enquanto o AG com os parâmetros base (artigo) e Distância Euclidiana como função fitness encontrou a solução em uma média de **50.74** gerações, o AG com parâmetros variados sugeridos por todos os integrantes do grupo convergiu em uma média de **45.50** gerações.

Em uma variação da função fitness para utilizar a Distância de Manhattan, o AG convergiu em uma média de **65.5** gerações.

